# 콘텐츠 기반 필터링 기법을 활용한 추천시스템

## 실습: TMDB 5000 영화 데이터 세트

### 데이터 불러오기

In [155]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [156]:
movie_df = pd.read_csv('./필요 데이터/tmdb_5000_movies.csv')
movie_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


### 데이터 전처리

#### 필요한 컬럼만 추출

In [157]:
movie_df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [158]:
movie = movie_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity',
                  'keywords', 'overview']]
movie.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


#### 딕셔너리형식 컬럼 수정

In [159]:
movie['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [160]:
movie['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [161]:
# 장르만 추출
movie['genres'] = movie['genres'].apply(lambda x: [dict_x['name'] for dict_x in eval(x)])

# 키워드만 추출
movie['keywords'] = movie['keywords'].apply(lambda x: [dict_x['name'] for dict_x in eval(x)])

movie.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca..."


### 장르 콘텐츠 유사도 측정

In [162]:
movie.shape

(4803, 8)

#### 리스트로 되어 있는 장르를 문자화하고 Count기반 피처 벡터화

In [163]:
from sklearn.feature_extraction.text import CountVectorizer

In [164]:
# 문자화
movie['genre_literal'] = movie['genres'].apply(lambda x: ' '.join(x))

# Count 기반 피처 벡터화
count_vect = CountVectorizer(min_df = 0, ngram_range = (1, 2))

count_features = count_vect.fit_transform(movie['genre_literal'])

count_features.shape

(4803, 276)

In [165]:
count_vect.get_feature_names()

['action',
 'action adventure',
 'action animation',
 'action comedy',
 'action crime',
 'action drama',
 'action family',
 'action fantasy',
 'action history',
 'action horror',
 'action mystery',
 'action romance',
 'action science',
 'action thriller',
 'action war',
 'action western',
 'adventure',
 'adventure action',
 'adventure animation',
 'adventure comedy',
 'adventure crime',
 'adventure documentary',
 'adventure drama',
 'adventure family',
 'adventure fantasy',
 'adventure history',
 'adventure horror',
 'adventure mystery',
 'adventure romance',
 'adventure science',
 'adventure thriller',
 'adventure war',
 'adventure western',
 'animation',
 'animation action',
 'animation adventure',
 'animation comedy',
 'animation drama',
 'animation family',
 'animation fantasy',
 'animation music',
 'animation mystery',
 'animation science',
 'animation war',
 'comedy',
 'comedy action',
 'comedy adventure',
 'comedy animation',
 'comedy crime',
 'comedy documentary',
 'comedy dram

#### 코사인유사도 측정

In [166]:
from sklearn.metrics.pairwise import cosine_similarity

In [167]:
cosine_sim = cosine_similarity(count_features, count_features)
cosine_sim.shape

(4803, 4803)

### 추천 시스템: 장르 유사도가 높고 평점이 높은 영화 추천

In [168]:
movie[movie['title'].duplicated() == True]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genre_literal
2877,1255,The Host,"[Horror, Drama, Science Fiction]",6.7,537,27.655270,"[river, mobile phone, bravery, archer, daughte...",Gang-du is a dim-witted man working at his fat...,Horror Drama Science Fiction
3693,10844,Out of the Blue,[Drama],5.9,18,0.706355,"[new zealand, gun rampage, police, independent...",Ordinary people find extraordinary courage in ...,Drama
4267,2661,Batman,"[Family, Adventure, Comedy, Science Fiction, C...",6.1,203,9.815394,"[submarine, dc comics, missile, shark attack, ...",The Dynamic Duo faces four super-villains who ...,Family Adventure Comedy Science Fiction Crime


In [169]:
import numpy as np

In [170]:
def find_similar_movie(title, movie_df, sim_array, num = 0, top_n = 10):
    # 찾고자하는 영화 인덱스 추출
    movie_idx = movie_df[movie_df['title'] == title].index[num]


    # 유사도가 높은 영화들의 내림차순 인덱스 추출
    sim_sort_idxes = sim_array[movie_idx, :].reshape(-1).argsort()[::-1]
    sim_sort_values = sim_array[movie_idx, sim_sort_idxes].tolist()

    
    # 추천 영화제목, 평점, 유사도 추출
    return_df = movie_df.loc[sim_sort_idxes,][['title', 'vote_average', 'vote_count']]
    return_df['similarity'] = sim_sort_values
    
    
    # 평점 순서대로 정렬
    return_df.sort_values(['similarity', 'vote_average'], ascending = False, inplace = True)
    
    return return_df.iloc[1:top_n + 1, :]
    

In [171]:
find_similar_movie('The Shawshank Redemption', movie, cosine_sim, top_n = 10)

,title,vote_average,vote_count,similarity
3337,The Godfather,8.4,5893,1.0
2731,The Godfather: Part II,8.3,3338,1.0
1847,GoodFellas,8.2,3128,1.0
1663,Once Upon a Time in America,8.2,1069,1.0
3866,City of God,8.1,1814,1.0
3887,Trainspotting,7.8,2655,1.0
892,Casino,7.8,1307,1.0
883,Catch Me If You Can,7.7,3795,1.0
4041,This Is England,7.4,363,1.0
281,American Gangster,7.4,1502,1.0


#### 투표횟수에 따른 가중치 부여 후 추천

In [174]:
def find_similar_movie(title, movie_df, sim_array, num = 0, top_n = 10):
    # 찾고자하는 영화 인덱스 추출
    movie_idx = movie_df[movie_df['title'] == title].index[num]
    
    # 가중치 주기 (v/(v + m )) * R + (m/(v + m)) * C
    C = movie_df['vote_average'].mean()
    m = movie_df['vote_count'].quantile(0.6)
    v = movie_df['vote_count']
    R = movie_df['vote_average']
    movie_df['weight'] = (v/(v + m )) * R + (m/(v + m)) * C
    

    # 유사도가 높은 영화들의 내림차순 인덱스 추출
    sim_sort_idxes = sim_array[movie_idx, :].reshape(-1).argsort()[::-1]
    sim_sort_values = sim_array[movie_idx, sim_sort_idxes].tolist()

    
    # 추천 영화제목, 평점, 유사도 추출
    return_df = movie_df.loc[sim_sort_idxes,][['title', 'vote_average', 'weight', 'vote_count']]
    return_df['similarity'] = sim_sort_values

    
    # 평점 순서대로 정렬
    return_df.sort_values(['similarity', 'weight', 'vote_average'], ascending = False, inplace = True)
    
    return return_df.iloc[1:top_n + 1, :]
    

In [175]:
find_similar_movie('The Shawshank Redemption', movie, cosine_sim, top_n = 10)

,title,vote_average,weight,vote_count,similarity
3337,The Godfather,8.4,8.263591,5893,1.0
2731,The Godfather: Part II,8.3,8.079586,3338,1.0
1847,GoodFellas,8.2,7.976937,3128,1.0
3866,City of God,8.1,7.759693,1814,1.0
1663,Once Upon a Time in America,8.2,7.657811,1069,1.0
3887,Trainspotting,7.8,7.591009,2655,1.0
883,Catch Me If You Can,7.7,7.557097,3795,1.0
892,Casino,7.8,7.423040,1307,1.0
281,American Gangster,7.4,7.141396,1502,1.0
4041,This Is England,7.4,6.739664,363,1.0


In [176]:
find_similar_movie('Spirited Away', movie, cosine_sim, top_n = 10)

,title,vote_average,weight,vote_count,similarity
92,How to Train Your Dragon,7.5,7.386631,4227,1.000000
90,The Polar Express,6.4,6.338207,1474,0.857143
54,The Good Dinosaur,6.6,6.510741,1736,0.845154
1680,Savva. Heart of the Warrior,6.4,6.097884,7,0.845154
399,Open Season,6.1,6.097176,656,0.845154
734,The Tale of Despereaux,5.8,5.972178,258,0.845154
141,Mars Needs Moms,5.5,5.885140,199,0.845154
187,Puss in Boots,6.4,6.261230,451,0.755929
106,Shrek the Third,6.0,6.012885,2278,0.755929
1078,The Ant Bully,5.5,5.797775,366,0.755929
